In [ ]:
%reload_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import hvplot.pandas  # noqa
pd.options.plotting.backend = 'holoviews'

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
# from mlxtend.feature_selection import ExhaustiveFeatureSelector as EFS
from sklearn.feature_selection import SelectKBest, mutual_info_regression, f_regression
from sklearn.model_selection import GridSearchCV, train_test_split, LeavePOut, LeaveOneOut

from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.preprocessing import StandardScaler, MaxAbsScaler, MinMaxScaler, OneHotEncoder, FunctionTransformer


from sklearn.dummy import DummyRegressor
from sklearn.ensemble import (
    RandomForestRegressor,
    AdaBoostRegressor,
    GradientBoostingRegressor,
)
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsRegressor, RadiusNeighborsRegressor
from sklearn.svm import SVR

from sklearn.metrics import r2_score

import prepare_data
from components import PCOA
from helpers import PipelineHelper, SMWrapper
from settings import Config, shortnames, target
from plots import scatter_chart

## Data preparation

In [ ]:
%%capture
# cell magic to supress output. Comment it out to see output of this cell.

# What happened so far: DB extract and blank procedure. Now import resulting MP data from csv
mp_pdd = prepare_data.get_pdd()

# Also import sediment data (sediment frequencies per size bin from master sizer export)
grainsize_iow, grainsize_cau = prepare_data.get_grainsizes()[0:2]
scor_iow = PCOA(grainsize_iow, 2)[0]
scor_cau = PCOA(grainsize_cau, 2)[0]

# ...some data wrangling to prepare particle domain data and sample domain data for MP and combine with certain sediment aggregates.
sdd_iow = prepare_data.aggregate_SDD(mp_pdd)
sdd_iow = prepare_data.additional_sdd_merging(sdd_iow, how='outer')
sdd_iow = sdd_iow.merge(scor_iow, right_index=True, left_on='Sample', how='outer')
sdd_iow = sdd_iow.replace({'Sample': shortnames}).sort_values(by='Sample')

sdd_cau = pd.read_csv('../data/Metadata_CAU_sampling_log.csv', index_col=0).join(pd.read_csv('../data/GRADISTAT_CAU_vol_log-cau_closed.csv', index_col=0), how='outer')
sdd_cau = sdd_cau.merge(scor_cau, right_index=True, left_on='Sample', how='outer').reset_index()

In [ ]:
## Additional variable generation (e.g. predictor derivatives)
 
# sdd_iow['Dist_WWTP_revsq'] = ((1/sdd_iow['Dist_WWTP'])**3)*10000000000  # calculates the squared of the reversed Distance 
# sdd_iow['Dist_WWTP_revsq'] = (((sdd_iow['Dist_WWTP'].max()-sdd_iow['Dist_WWTP'])+1)**3)/100000000000  # calculates the squared of the reversed Distance 
# sdd_iow['Dist_WWTP_revsq'] = ((sdd_iow['Dist_WWTP'].max()/sdd_iow['Dist_WWTP'])**3)/100  # calculates the squared of the reversed Distance
sdd_iow['Dist_WWTP_revsq'] = ((sdd_iow['Dist_WWTP'].max()/sdd_iow['Dist_WWTP'])**2)  # calculates the squared of the reversed Distance

# sdd_iow

In [ ]:
## Split data into samples used for building the model and samples used for predicting.

model_data = sdd_iow.loc[~sdd_iow.Concentration.isna()].set_index('Sample')
pred_data = sdd_iow.loc[sdd_iow.Concentration.isna()]
pred_data = pd.concat([pred_data, sdd_cau.drop('Date',axis=1)]).set_index('Sample')

In [ ]:
## Potential outlier exclusion
model_data = model_data.drop(['S08','S10d','S05','S32'])

In [ ]:
## List of features (predictors) to be used in the model. Beware: depending on the preprocessing steps not all features might be used.

featurelist = [
    #'Depth',
    # 'LON', 'LAT',
    #'Dist_Land',
    # 'Dist_Marina',
    #'Dist_WWTP',
    # 'Dist_WWTP2',
    'Dist_WWTP_revsq',
    # 'MODE 1 (µm)',
    # 'D10 (µm)',
    #'D50 (µm)',
    # 'D90 (µm)',
    # 'perc GRAVEL',
    # 'perc SAND',
    'perc MUD',
    # 'perc CLAY',
    # 'OM_D50',
    'TOC',
    # 'Hg',
    # 'TIC',
    # 'regio_sep',
    # 'PC1', 'PC2'
    ]
model_X = model_data[featurelist]
model_y = model_data[target]
pred_X = pred_data[featurelist]

In [ ]:
## Mutual exclusive list (list of lists detailing predictors that are not allowed together in one model candidate)

mutual_exclusive = [
    ['D50 (µm)','PC1','perc MUD'],
    ['Dist_WWTP','Dist_WWTP2'],
    ['TOC', 'perc MUD'],
]

In [ ]:
## Check some basic statistics of the target variable

model_y.describe()
# model_y.hist()


## Model building

### Custom preprocessing functions to be used in the model pipeline

In [ ]:
## Create exhaustive feature selector, using leave-p-out on columns labels to generate a boolean matrix.

feature_candidates_list = []

for i in range(1,len(model_X.columns)):
    lpo = LeavePOut(p=i)
    # base_index = [False] * len(model_X.columns)
    for candidate_indices, _ in lpo.split(model_X.columns):
        feature_candidates = model_X.columns[candidate_indices]
        if any(all(pd.Series(ex_feats).isin(feature_candidates)) for ex_feats in mutual_exclusive):
            continue  # if all entries of any row in mutual_exclusive list are present in the current feature_candidates, then don't put them in the feature_candidates_list
        feature_candidates_list.append(feature_candidates)
feature_candidates_list.append(model_X.columns)  # also append the set of all possible features

def SelectFeatures(model_X, feature_set=0, feature_sets=feature_candidates_list):
    return model_X.loc[:, feature_sets[feature_set]]

CustomFeatureSelector = FunctionTransformer(SelectFeatures, feature_names_out='one-to-one')

### Creating the model pipeline

In [ ]:
## Pipeline and parameter grid for model selection, see here for inspiration: https://towardsdatascience.com/getting-the-most-out-of-scikit-learn-pipelines-c2afc4410f1a

PreProcessor = ColumnTransformer([
      ('selector', CustomFeatureSelector, model_X.columns),
      # ('imputer', SimpleImputer(), make_column_selector(dtype_include=np.number)),
      ('scaler', StandardScaler(), make_column_selector(dtype_include=np.number)),
      # ('encoder', OneHotEncoder(), make_column_selector(dtype_include=object)),
      ])

pipe = Pipeline(steps=[
    ('preprocessor', PreProcessor),
    ('regressor', DummyRegressor())
    ])

params = [{
    'preprocessor__selector': [CustomFeatureSelector],
    'preprocessor__selector__kw_args': [{'feature_set':i} for i in range(len(feature_candidates_list))]
    },
    
    {'preprocessor__scaler': [StandardScaler()],
       'preprocessor__scaler__with_mean': [True, False],
       'preprocessor__scaler__with_std': [True],#, False],
    },

    {'preprocessor__scaler': [MaxAbsScaler()]
    },

    {'preprocessor__scaler': [MinMaxScaler()]
    },

    {'regressor': [DummyRegressor()],
        'regressor__strategy': ['mean', 'median']
    },
    
    {'regressor': [SVR()],
        'regressor__C': [1], #[0.1, 0.5, 1.0],
        'regressor__kernel': ['linear'], #['linear', 'rbf', 'poly'],
        'regressor__degree': [1] #[1, 2, 3, 4, 5],
    },
    
    {'regressor': [RandomForestRegressor()],
        'regressor__n_estimators': [10, 50], #[20, 50, 100, 150],
        'regressor__max_features': [None, 'sqrt', 'log2'],
        'regressor__min_samples_split': [2, 5, 10],
        'regressor__min_samples_leaf': [1, 2, 5],
        'regressor__bootstrap': [True, False],
        'regressor__max_depth': [None, 5, 10, 20, 50],
        'regressor__warm_start': [True, False]
    }
    ]

### Training the model

In [ ]:
## The pipeline is run by searching the provided paramter space using scorings of a crossvalidation technique to find out how each model candidate performs.

scoring = ['neg_mean_absolute_error']  # possibilities: ‘neg_root_mean_squared_error’, ‘neg_mean_squared_error’, 'r2', 'neg_mean_absolute_error', 'neg_mean_squared_log_error'
    

grid = GridSearchCV(  # TODO: Possible to set random state for all estimators?
    pipe,
    params,
    scoring= scoring,
    error_score=np.nan,
    refit= scoring[0],
    cv=LeaveOneOut(),
    verbose=1,
    n_jobs=-1
    )

grid.fit(model_X, model_y)

## Evaluating the model

In [ ]:
## Printing score of the best performing model candidate and its parameters.

print(f'{scoring}: {grid.best_score_}')
print(grid.best_params_)

In [ ]:
[grid.best_estimator_.named_steps['preprocessor'].named_transformers_['selector'].get_feature_names_out(input_features=model_X.columns.tolist())]

In [ ]:
## Show how the model performs on the training data

train_pred_y = grid.predict(model_X)  # use the best model to predict the data on the same samples that were used to train the model
print(f'R² = {r2_score(model_y, train_pred_y)}')

df = pd.concat([
    model_data.loc[model_y.index].regio_sep,
    model_y,
    pd.Series(
        train_pred_y,
        name='Prediction',
        index=model_y.index)
        ],
    axis=1
    ).reset_index()
    
scatter_chart(df, target, 'Prediction', color='regio_sep', labels='Sample', identity=True, equal_axes=False)[0]

In [ ]:
## Take a look at all model candidates and their performance

scores = pd.DataFrame(grid.cv_results_)
# scores